In [ ]:
%load_ext ipybind
%matplotlib inline

# xtensor

[xtensor](http://xtensor.readthedocs.io) is an array library in C++.  It defines the multi-dimensional array data structure suitable for compile-time optimization.

1. xtensor: array library in C++
2. Carry arrays between Python and C++
3. Speed up array-based Python API using C++

In [ ]:
import warnings
warnings.filterwarnings('default')

import cxxfilt
import numpy as np

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(12, 8))

# Major source of overhead: data preparation

Demonstration with polynomial curve fitting for data in groups of variable length.

In [ ]:
%%time
xdata = np.unique(np.random.sample(1000000) * 1000) # the unique return sorted result
ydata = np.random.sample(len(xdata)) * 1000

In [ ]:
%%time
data_groups = []
for i in range(1000):
    slct = (xdata>=i)&(xdata<(i+1))
    data_groups.append((xdata[slct], ydata[slct]))

In [ ]:
%%pybind11

#include "pybind11/pybind11.h"
#define FORCE_IMPORT_ARRAY
#include "xtensor-python/pyarray.hpp"

#include <vector>
#include <algorithm>

#include "xtensor/xarray.hpp"
#include "xtensor/xadapt.hpp"
#include "xtensor/xview.hpp"
#include "xtensor-blas/xlinalg.hpp"

using array_type = xt::xarray<double>;
using view_type = xt::xview<array_type&, xt::xrange<long>>;

template <class AT>
xt::xarray<double> fit_poly(AT & xarr, AT & yarr, size_t order)
{
    if (xarr.size() != yarr.size()) { throw std::runtime_error("xarr and yarr size mismatch"); }
    
    xt::xarray<double> matrix(std::vector<size_t>{order+1, order+1});

    for (size_t it=0; it<order+1; ++it)
    {
        for (size_t jt=0; jt<order+1; ++jt)
        {
            double & val = matrix(it, jt);
            val = 0;
            for (size_t kt=0; kt<xarr.size(); ++kt) { val += pow(xarr[kt], it+jt); }
        }
    }

    xt::xarray<double> rhs(std::vector<size_t>{order+1});
    for (size_t jt=0; jt<order+1; ++jt)
    {
        rhs[jt] = 0;
        for (size_t kt=0; kt<yarr.size(); ++kt) { rhs[jt] += pow(xarr[kt], jt) * yarr[kt]; }
    }

    xt::xarray<double> lhs = xt::linalg::solve(matrix, rhs);
    std::reverse(lhs.begin(), lhs.end()); // to make numpy.poly1d happy.

    return lhs;
}

template <class AT>
xt::xarray<double> fit_polys(xt::xarray<double> & xarr, xt::xarray<double> & yarr, size_t order)
{
    size_t xmin = std::floor(*std::min_element(xarr.begin(), xarr.end()));
    size_t xmax = std::ceil(*std::max_element(xarr.begin(), xarr.end()));
    size_t ninterval = xmax - xmin;

    xt::xarray<double> lhs(std::vector<size_t>{ninterval, order+1});
    lhs.fill(0); // sentinel.
    size_t start=0;
    for (size_t it=0; it<xmax; ++it)
    {
        // Take advantage of the input being sorted.
        size_t stop;
        for (stop=start; stop<xarr.size(); ++stop) { if (xarr[stop]>=it+1) { break; } }

        AT sub_x = xt::view(xarr, xt::range(start, stop));
        AT sub_y = xt::view(yarr, xt::range(start, stop));        

        xt::xarray<double> sub_lhs = fit_poly(sub_x, sub_y, order);
        xt::view(lhs, it, xt::all()) = sub_lhs;
        
        start = stop;
    }

    return lhs;
}

PYBIND11_MODULE(example, m)
{
    xt::import_numpy();
    m.def
    (
        "fit_poly"
      , [](xt::pyarray<double> & xarr_in, xt::pyarray<double> & yarr_in, size_t order)
        {
            std::vector<size_t> xarr_shape(xarr_in.shape().begin(), xarr_in.shape().end());
            xt::xarray<double> xarr = xt::adapt(xarr_in.data(), xarr_shape);

            std::vector<size_t> yarr_shape(yarr_in.shape().begin(), yarr_in.shape().end());
            xt::xarray<double> yarr = xt::adapt(yarr_in.data(), yarr_shape);

            return fit_poly(xarr, yarr, order);
        }
    );
    m.def
    (
        "fit_polys_array"
      , [](xt::pyarray<double> & xarr_in, xt::pyarray<double> & yarr_in, size_t order)
        {
            std::vector<size_t> xarr_shape(xarr_in.shape().begin(), xarr_in.shape().end());
            xt::xarray<double> xarr = xt::adapt(xarr_in.data(), xarr_shape);
            std::vector<size_t> yarr_shape(yarr_in.shape().begin(), yarr_in.shape().end());
            xt::xarray<double> yarr = xt::adapt(yarr_in.data(), yarr_shape);
            return fit_polys<array_type>(xarr, yarr, order);
        }
    );
    m.def
    (
        "fit_polys_view"
      , [](xt::pyarray<double> & xarr_in, xt::pyarray<double> & yarr_in, size_t order)
        {
            std::vector<size_t> xarr_shape(xarr_in.shape().begin(), xarr_in.shape().end());
            xt::xarray<double> xarr = xt::adapt(xarr_in.data(), xarr_shape);
            std::vector<size_t> yarr_shape(yarr_in.shape().begin(), yarr_in.shape().end());
            xt::xarray<double> yarr = xt::adapt(yarr_in.data(), yarr_shape);
            return fit_polys<view_type>(xarr, yarr, order);
        }
    );
    m.attr("fit_polys") = m.attr("fit_polys_array");
}

In [ ]:
%%time
polygroup = np.empty((len(data_groups), 3), dtype='float64')
for i in range(1000):
    slct = (xdata>=i)&(xdata<(i+1))
    sub_x = xdata[slct]
    sub_y = ydata[slct]
    polygroup[i,:] = fit_poly(sub_x, sub_y, 2)

In [ ]:
%%time
polygroup = np.empty((len(data_groups), 3), dtype='float64')
for it, (sub_x, sub_y) in enumerate(data_groups):
    polygroup[it,:] = fit_poly(sub_x, sub_y, 2)

In [ ]:
%%time
allpoly = fit_poly(xdata, ydata, 2)

In [ ]:
print(allpoly)
poly = np.poly1d(allpoly)
xp = np.linspace(xdata.min(), xdata.max(), 100)
plt.plot(xdata, ydata, '.', xp, poly(xp), '-')

In [ ]:
%%time
rbatch = fit_polys(xdata, ydata, 2)

In [ ]:
%%time
rbatch = fit_polys_array(xdata, ydata, 2)

In [ ]:
%%time
rbatch = fit_polys_view(xdata, ydata, 2)

In [ ]:
rbatch = fit_polys(xdata, ydata, 2)
print(rbatch.shape)
# Verify batch.
for i in range(1000):
    assert (rbatch[i] == polygroup[i]).all()

In [ ]:
i = 0
slct = (xdata>=i)&(xdata<(i+1))
sub_x = xdata[slct]
sub_y = ydata[slct]
poly = fit_poly(sub_x, sub_y, 3)
print(poly)
#poly = np.polyfit(sub_x, sub_y, 3)
#poly = np.polynomial.polynomial.polyfit(sub_x, sub_y, 3)
#print(poly)
poly = np.poly1d(poly)
xp = np.linspace(sub_x.min(), sub_x.max(), 100)
plt.plot(sub_x, sub_y, '.', xp, poly(xp), '-')

# `xarray` and `xview`

In [ ]:
%%pybind11

#include "pybind11/pybind11.h"
#define FORCE_IMPORT_ARRAY
#include "xtensor-python/pyarray.hpp"

#include <vector>
#include <algorithm>

#include "xtensor/xarray.hpp"
#include "xtensor/xview.hpp"

using array_type = xt::pyarray<double>;
using view_type = xt::xview<array_type&, xt::xrange<long>>;

void set_view(xt::pyarray<double> & arr, size_t start, size_t stop, double fill_value)
{
    view_type view = xt::view(arr, xt::range(start, stop));
    view.fill(fill_value);
}

PYBIND11_MODULE(example, m)
{
    xt::import_numpy();
    m.def("set_view", set_view);
    m.attr("view_type_name") = typeid(view_type).name();
}

In [ ]:
print(cxxfilt.demangle(view_type_name, external_only=False))

arr = np.arange(10, dtype='float64')
print(arr)
set_view(arr, 3, 5, -1)
print(arr)

# `pyarray` and `xarray`

Todo:

1. Elemental operations in C++ runs much faster than that in Python
2. Static and dynamic arrays with xtensor
3. Algorithms are easier to be described in elemental mode, but house-keeping code is easier to be done in batch mode.
4. Find 4 algorithms to present the use of xtensor.  One for lecture and three for exercises.
  1. Curve fitting.  Least square.

# Problems

1. By allowing changing the signature of the `fit_poly` function, how can we ensure the shapes of `xarr` and `yarr` to be the same, without the explicit check with `"xarr and yarr size mismatch"`?  Write code to show.